In [1]:
from rdkit import Chem
from rdkit.Chem.rdMolTransforms import ComputeCentroid
from pathlib import Path
lig = Chem.SDMolSupplier('../1iep_ligandH.sdf', removeHs=False)[0]
centroid = ComputeCentroid(lig.GetConformer())

from vina import Vina

import glob
lig_files = glob.glob('../data/ligands/*.pdbqt')
print(lig_files)

['../data/ligands/ZINC00000179.pdbqt', '../data/ligands/imatinib.pdbqt', '../data/ligands/ZINC00000570.pdbqt', '../data/ligands/ZINC00000171.pdbqt']


In [2]:
print(centroid.x, centroid.y, centroid.z)

15.6138918918919 53.38013513513513 15.45483783783784


In [3]:
for lig in lig_files:
    fname = Path(lig).stem
    v = Vina(sf_name='vina')
    v.set_receptor('../generatedfile/1iep_receptor.pdbqt')
    v.set_ligand_from_file(lig)
    v.compute_vina_maps(center=[centroid.x, centroid.y, centroid.z], box_size=[20, 20, 20])
    energy_minimized = v.optimize()
    print('Score after minimization : %.3f (kcal/mol)' % energy_minimized[0])
    v.write_pose(f'{fname}_minimized.pdbqt', overwrite=True)
    v.dock(exhaustiveness=8, n_poses=5)
    v.write_poses(f'1iep_{fname}_vina_out.pdbqt', n_poses=10, overwrite=True)

Score after minimization : 1006320659.702 (kcal/mol)
Score after minimization : 910074457.314 (kcal/mol)
Score after minimization : 486005145.186 (kcal/mol)
Score after minimization : 852214995.626 (kcal/mol)


In [4]:
from pymol import cmd
receptor = '../generatedfile/1iep_receptor.pdbqt'
docked_ligands = glob.glob('1iep_ZINC*_vina_out.pdbqt')
docked_ligands.append('1iep_imatinib_vina_out.pdbqt')

print(docked_ligands)

['1iep_ZINC00000171_vina_out.pdbqt', '1iep_ZINC00000179_vina_out.pdbqt', '1iep_ZINC00000570_vina_out.pdbqt', '1iep_imatinib_vina_out.pdbqt']


In [5]:
cmd.delete('all')
cmd.load(receptor)
for l in docked_ligands:
    cmd.load(l)
    target_obj = l.replace('.pdbqt', '')
    cmd.split_states(target_obj)
    cmd.delete(target_obj)
    cmd.group(target_obj, target_obj+'*')
cmd.save('vina_docked_complex.pse')
